## NOTEBOOK 1 ##
Group members:

150487 Kevin Otieno

169423 Ryan Matu

134706 Peter Wachira

169957-Joshua Munyirwa

122830 Princess Imelda Sidai

168956 Mutunga Eric Musyimi

# Task Selection and Justification

**Selected Task:** Regression

**Justification:**
The regression task is the most suitable for this dataset because our goal is to predict the `hg/ha_yield` (Maize yield), which is a continuous numerical variable.

* **Nature of Data:** We are predicting a specific quantity (amount of crop per hectare) rather than a class label.
* **Why not Classification?** Classification would be appropriate if we were predicting discrete categories (e.g., "High Yield" vs. "Low Yield"). Since we need to estimate the exact numerical yield based on inputs like rainfall and temperature, regression algorithms (like Random Forest Regressor) are the correct choice.

In [ ]:
import pandas as pd
import numpy as np
import io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
url = 'https://raw.githubusercontent.com/OKevina/AI-assignment-dataset/refs/heads/main/maize_global_cleaned.csv'

In [ ]:
df = pd.read_csv(url)

In [ ]:
df.head()

,Unnamed: 0,Country,Crop,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp
0,0,Albania,Maize,1990,36613,1485.0,121.0,16.37
1,6,Albania,Maize,1991,29068,1485.0,121.0,15.36
2,12,Albania,Maize,1992,24876,1485.0,121.0,16.06
3,18,Albania,Maize,1993,24185,1485.0,121.0,16.05
4,23,Albania,Maize,1994,25848,1485.0,201.0,16.96


In [ ]:
df.describe()

,Unnamed: 0,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp
count,4121.000000,4121.000000,4121.000000,4121.000000,4121.000000,4121.000000
mean,14135.846639,2001.553749,36310.070614,1098.124242,32765.983322,19.925159
std,8201.607601,7.044490,27456.370877,721.559071,54088.622824,6.654389
min,0.000000,1990.000000,849.000000,51.000000,0.040000,1.610000
25%,6847.000000,1995.000000,17086.000000,537.000000,1597.000000,15.670000
50%,14595.000000,2001.000000,25401.000000,1020.000000,14485.330000,20.810000
75%,21228.000000,2008.000000,48243.000000,1622.000000,43720.040000,25.920000
max,28235.000000,2013.000000,207556.000000,3240.000000,367778.000000,30.650000


In [ ]:
target = 'hg/ha_yield' # Defining the target variable for prediction

In [ ]:
numeric_features = ['Year', 'average_rain_fall_mm_per_year', 'pesticides_tonnes', 'avg_temp'] # Defining numeric features
categorical_features = ['Country'] # Defining categorical features

In [ ]:
X = df[numeric_features + categorical_features] # Creating feature matrix X
y = df[target] # Creating target vector y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Splitting data into training and testing sets

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features), # Applies StandardScaler to numeric features
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features) # Applies OneHotEncoder to categorical features
    ])

In [ ]:
model = Pipeline([
    ('preprocessor', preprocessor), # Applies the defined preprocessor to the data
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42)) # Uses a RandomForestRegressor for the prediction task
])

In [ ]:
model.fit(X_train, y_train) # Training the model with the training data

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Year',
                                                   'average_rain_fall_mm_per_year',
                                                   'pesticides_tonnes',
                                                   'avg_temp']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Country'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [ ]:
y_pred = model.predict(X_test)

metrics = {
    'R2 Score': r2_score(y_test, y_pred),
    'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
    'MAE': mean_absolute_error(y_test, y_pred)
}

In [ ]:
pd.DataFrame([metrics], index=['Random Forest'])

,R2 Score,RMSE,MAE
Random Forest,0.964744,5088.041932,2315.260412


In [ ]:
# Extract feature names
ohe_feature_names = model.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features)
all_feature_names = numeric_features + list(ohe_feature_names)

# Get importances
importances = model.named_steps['regressor'].feature_importances_

# Create DataFrame and display top 10
feature_importance_df = pd.DataFrame({'feature': all_feature_names, 'importance': importances})
feature_importance_df.sort_values(by='importance', ascending=False).head(10)

,feature,importance
3,avg_temp,0.352575
1,average_rain_fall_mm_per_year,0.100339
76,Country_Qatar,0.095707
2,pesticides_tonnes,0.093839
0,Year,0.067418
83,Country_Spain,0.062011
36,Country_Greece,0.040887
48,Country_Japan,0.037016
9,Country_Australia,0.021408
30,Country_Egypt,0.016139


#  Model Evaluation and Interpretation

The model was evaluated using R² Score, Root Mean Squared Error (RMSE), and Mean Absolute Error (MAE).

### **Results Interpretation:**

* **R² Score (~0.96):**
    The model explains approximately 96.5% of the variance in the maize yield. This is an exceptionally high score, indicating that the selected features (Rainfall, Temperature, Country, etc.) are very strong predictors of crop production.

* **Error Margins (MAE & RMSE):**
    * **MAE (~2,315 hg/ha):** On average, the model's predictions deviate from the actual yield by about 2,315 hectograms per hectare.
    * **Context:** Given that the average yield in the dataset is roughly 36,000 hg/ha, an error of ~2,300 represents a deviation of less than 10%, making the model reliable for general estimation.

### **Feature Importance:**

The analysis highlights the key drivers of maize yield:
1.  **Average Temperature (`avg_temp`):** This was identified as the most critical factor, confirming that maize is highly sensitive to thermal conditions.
2.  **Rainfall (`average_rain_fall_mm_per_year`):** Water availability is the second most important environmental predictor.
3.  **Country Effects:** The high importance of specific "Country" features suggests that local factors (soil quality, farming technology, policy) play a massive role in yield differences, distinct from just weather patterns.